In [0]:
from torchvision.transforms import transforms

In [0]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomAffine(20, translate=(0.15, 0.15)),
    transforms.ToTensor()
])

In [0]:
from torch.utils.data import Dataset 

# transform 까지 추가해준다
class BengaliDataset(Dataset):
    def __init__(self, csv, img_height, img_width, transform):
        self.csv = csv.reset_index()
        self.img_ids = csv['image_id'].to_numpy()
        self.img_height = img_height
        self.img_width = img_width
        self.transform = transform

    def __len__(self):
        return len(csv)

    def __getitem__(self, index):
        img_id = self.img_ids[index]
        img = joblib.load(f'data/train_image/{img_id}.pkl')
        img = img.reshape(self.img_height, self.img_width).astype(np.uint8)
        img = 255 - img
        img = img[:, :, np.newaxis]
        img = np.repeat(img, 3, 2)
        if self.transform is not None:
            img = self.transform(image=img) # albumentations과 다른 부분
        
        label_1 = self.csv.iloc[index].grapheme_root
        label_2 = self.csv.iloc[index].vowel_diacritic
        label_3 = self.csv.iloc[index].consonant_diacritic

        return img, label_1, label_2, label_3